In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('/content/drive/MyDrive/datasets/subtask_2/es/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text label
0  12786  Sin embargo, los jóvenes son capaces de recono...     B
1  12361  ¿Hay algo más que quieras compartir? ¿Algo sob...     B
2   1662  El servicio de sala es bueno, rápido y amabilí...     B
3  14729  Para concentrarse en el hablante, trata de des...     F
4   9312  Los responsables locales tendrán ahora que esp...     F


In [19]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
print("Unique labels: ", set(train_data_labels))


Unique labels:  {'A', 'D', 'E', 'F', 'C', 'B'}


In [20]:
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))

train data size:  17766
validation data size:  1975
test data size:  2194


In [21]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [22]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels[idx] == 'A':
            item['labels'] = torch.tensor(0)
        elif self.labels[idx] == 'B':
            item['labels'] = torch.tensor(1)
        elif self.labels[idx] == 'C':
            item['labels'] = torch.tensor(2)
        elif self.labels[idx] == 'D':
            item['labels'] = torch.tensor(3)
        elif self.labels[idx] == 'E':
            item['labels'] = torch.tensor(4)
        elif self.labels[idx] == 'F':
            item['labels'] = torch.tensor(5)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

print("Sample Data Point")
print()
print(train_dataset[0])

Sample Data Point

{'input_ids': tensor([    4,  3788,  1468,  1084,  2778,  1038, 29910, 11484,  1397,  1110,
         1200,  1808,  1698,  1017,  2825,  1216,  1311,  1091,  5430,  1181,
        29414,  1030,  3511,  1008,  3617,  2114,  2744,  1042, 14330,  1252,
         1294,  4116,  1009,  1067, 16108,  1038,  5569,  3166,  3757,  1008,
         2360,  1064,  1278,  2756,  1017,  1089,  1216, 14159, 30934,  1404,
         8587,  5845,  1051,  1030,  4646,  1074,  1051,  2259,  1932,  1008,
        10473,  9532,  1009,  1448,  1447,  1423,  2074,  7657,  1789,  1216,
        18135,  1042, 18904, 30934,  1009,  1187,  1365,  3141,  1017,  5334,
         7119,  2859,  3487,  1038,  1835,  3189,  5316,  1007,  6764,  1110,
         1248,  1009,  1261,  1359,  1785,  1017, 11650,     5,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,   

In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [24]:
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased",num_labels=6).to(device)
print("Model Configurations")
print()
print(model.config)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Model Configurations

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [25]:
training_args = TrainingArguments(
    output_dir='./results',       
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.034800,1.182669,0.491139,0.422403,0.452095,0.496969
2,0.762000,0.952738,0.571646,0.565239,0.573240,0.568154
3,0.512600,1.004478,0.581772,0.579416,0.599273,0.581785


TrainOutput(global_step=1668, training_loss=0.8810255861968446, metrics={'train_runtime': 1955.0572, 'train_samples_per_second': 27.262, 'train_steps_per_second': 0.853, 'total_flos': 6080630585880816.0, 'train_loss': 0.8810255861968446, 'epoch': 3.0})

In [26]:
trainer.evaluate(test_dataset)
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.77      0.68      0.72       339
           1       0.43      0.48      0.45       335
           2       0.50      0.54      0.52       357
           3       0.76      0.40      0.52       367
           4       0.56      0.70      0.62       399
           5       0.81      0.93      0.87       397

    accuracy                           0.63      2194
   macro avg       0.64      0.62      0.62      2194
weighted avg       0.64      0.63      0.62      2194

